# Executive summary

This study investigates the <a href="#intro"> gender pay gap for US Kagglers in 2020</a>. 

We show that the gender pay gap can be decomposed into two contributing factors: 

1. <a href="#factor1">There are fewer women with higher-paying background and experience.</a> 

2. <a href="#factor2">Women are paid less than peers with the same background and experience.</a> 

This decomposition of dual factors offers insights on <a href="#mitigate">how to mitigate the gender pay gap</a>. 

We conclude with some <a href="#future">recommendations for future Kaggle surveys</a> that could aid further studies on the gender pay gap. 

In [ ]:
### Data setup and cleaning

# Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for plotting

# Load data
data = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

# Remove row of questions
questions = data.iloc[0]
questions.head()
data.drop(0, inplace=True)

# Remove respondent #11463, who claims to be in age group 18-21, has 20+ years of programming experience ("I was coding before I could talk"?!), and earns >$500,000
data.drop(11463, inplace=True)

# Create binary column indicating whether or not respondent is a woman
data['is_woman'] = data.Q2 == 'Woman'

# Create column of estimated average salary
data['salary'] = data.Q24.str.replace('$','').str.replace('>','').str.replace(',','').str.split('-').apply(lambda x: (int(x[0]) + int(x[-1]))/2 if type(x)==list else x)
# To perform the analysis with median salary, use the following code to obtain the lower bound on the salary range, and change "mean" to "median" in all code below
#data['salary'] = data.Q24.str.replace('$','').str.replace('>','').str.replace(',','').str.split('-').apply(lambda x: int(x[0]) if type(x)==list else x)

# Extract US data
us = data[(data.Q3=='United States of America')]

<section id="intro">
<h1>The gender pay gap for 2020 US Kagglers</h1>
</section>

Based on the [2020 Kaggle ML & DS Survey](https://www.kaggle.com/c/kaggle-survey-2020), for US Kagglers in 2020, the average annual salary ratio between women and non-women is 96.89/125.44 = 77%. This is lower than the 2019 US female-to-male earnings ratio of 82.3%, [as reported by the US Census Bureau](https://www.census.gov/library/publications/2020/demo/p60-270.html).

In [ ]:
### Plot US gender pay gap

gender_salary = us.groupby('is_woman').salary.mean()

ax = gender_salary.plot.barh()

ax.set_title('Average annual salary for 2020 US Kagglers', fontsize=15)

ax.set_xlabel('USD')
ax.set_xticklabels([str(int(x/1000))+'k' for x in ax.get_xticks()])
ax.set_xlim((0, gender_salary.max()*1.3))

ax.set_ylabel('')
ax.set_yticklabels(['Non-women', 'Women'], fontsize=15)

for p in ax.patches:
    ax.annotate('$' + str(round(p.get_width()/1000, 2)) + 'k', (p.get_width()*1.02, p.get_y()+0.2), fontsize=15)

plt.show()

This is a well-documented phenomenon known as the [gender pay gap](https://en.wikipedia.org/wiki/Gender_pay_gap). 

In this notebook, we show that the gender pay gap for 2020 US Kagglers can be decomposed into two contributing factors: 

1. There is a lower percentage of women with higher-paying background and experience, such as years in programming or job level; 

2. Even when controlled for background and experience, women were still paid less than non-women. 

Before proceeding, we make a couple of remarks on our methodology: 

- In this study, **background and experience** are broadly defined to include as many attributes as we could glean from survey data. "Background" includes job title, age, education, and attributes of their company; "experience" includes years in programming or in machine learning, as well as the number of technical tools used on a regular basis. 

- The **average annual salary** is an "estimated average" that is calculated as follows: Each salary range is represented by the midway point of the range (for example, `$0-999` is replaced with (0+999)/2=499.5, `1,000-1,999` is replaced with (1000+1999)/2 = 1499.5, and so on), and the highest range, `> $500,000`, is replaced with the lower bound of 500000. This differs from the conventional approach of using the median salary. The estimated average appears to be more informative because it provides more granularity, whereas the median salary is more coarse due the limited choices of salary range offered in the survey. That said, our code can easily be modified to use the median salary instead; see comments in the first code block of the notebook. 

- We compare women with **non-women**, which combines all other choices offered in the survey, including "Man" (75.10%), "Prefer not to say" (2.24%), "Prefer to self-describe" (0.27%), and "Nonbinary" (0.80%), for a total of 78.41% US survey respondents. Women accounted for the remaining 21.59%.

In [ ]:
### Calculate US gender pay gap
round(gender_salary[True] / gender_salary[False] * 100, 2)

In [ ]:
### Calculate percentage of respondents by gender
print(round(us.Q2.value_counts() / us.Q2.shape[0] * 100, 2))
print('Percentage of non-women: ', round((1-us.is_woman.mean())*100, 2))

<section id="factor1">
<h1>Factor one: Fewer women with higher-paying background and experience</h1>
</section>

There is a lower percentage of women with higher-paying background and experience. 

In [ ]:
### Function for plotting salary and percentage of women by background/experience

def plot_salary_and_percent_women(col_name, col_title, row_order): 
    '''
    Plot salary and percentage of women for a given column. 
    '''
    ax = us.groupby(col_name).agg({'salary': np.mean, 'is_woman': np.mean}).reindex(row_order).dropna()\
        .plot.barh(subplots=True, layout=(1,2), sharex=False, sharey=True, legend=False, figsize=(10,5))

    ax[0,0].set_xlabel('USD')
    ax[0,0].set_ylabel(col_title, fontsize=15)
    ax[0,0].set_title('Salary', fontsize=15)
    ax[0,0].set_xticklabels([str(int(x/1000))+'k' for x in ax[0,0].get_xticks()])
    
    ax[0,1].set_title('Percent women', fontsize=15)
    ax[0,1].set_xticklabels([str(int(x*100))+'%' for x in ax[0,1].get_xticks()])

For example, as we move up in years of programming experience (survey Q6), the salary generally increases, while the percentage of women decreases: 

In [ ]:
### Plot salary and percent women by programming experience
experience_ordered = ['I have never written code', '< 1 years', '1-2 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'] # ordering of experience levels
plot_salary_and_percent_women('Q6', 'Programming Experience', experience_ordered)

To a lesser extent, this effect also exists by job title (survey Q5). To bring this point to focus, we aggregate job titles by salary as follows: 

In [ ]:
### Aggregate job title by salary level

# list of salary levels
level = ['Level ' + str(i) for i in range(5)]

# dictionary that maps title to salary level
title2level = {'Student': level[0], 'Currently not employed': level[0], 
             'Business Analyst': level[1], 'Data Analyst': level[1], 
             'Research Scientist': level[2], 'Statistician': level[2], 'DBA/Database Engineer': level[2], 'Other': level[2], 
             'Data Engineer': level[3], 'Product/Project Manager': level[3], 'Software Engineer': level[3], 
             'Machine Learning Engineer': level[4], 'Data Scientist': level[4]
            }

# create column of salary level
us['job_level'] = us.Q5.apply(lambda x: title2level[x] if pd.isna(x)==False else x) # create column with job level

In [ ]:
### Plot job title and job level (aggregated by salary)

# job title
ax = us.groupby('Q5').salary.mean().dropna().sort_values().plot.barh()
ax.set_title('Salary by Job Title', fontsize=15)
ax.set_ylabel('')
ax.set_xticklabels([str(int(x/1000))+'k' for x in ax.get_xticks()])
ax.set_xlim(0, 200000)
ax.set_xlabel('USD')

# job level squares and labels
positions = [(-0.5, 95000, 2), (1.5, 120000, 4), (5.5, 135000, 3), (8.5, 150000, 2)]
for i, (ymin, xlen, yheight) in enumerate(positions): 
    rectangle = plt.Rectangle((0, ymin), xlen, yheight, fc='none', ec="red", linewidth=3)
    ax.add_patch(rectangle)
    ax.text(xlen+3000, ymin+yheight/2-0.25, 'Level '+str(i+1), color='red', fontsize=15)

plt.show()

For job levels 1-3 (which account for the majority of the paid population*, 69%), the higher the job level, the fewer women there are: 

In [ ]:
### Plot salary and percent women for each experience level
plot_salary_and_percent_women('job_level', 'Job Level', level)

In [ ]:
### Percentage of population for job levels 1-3 versus
round(us.job_level.isin(['Level 1', 'Level 2', 'Level 3']).sum() / (us.job_level != 'Level 0').sum() * 100, 2)

**Note on "paid population": The survey apparently does not ask for the salary of those who selected the option of "Student" or "Currently not employed". Thus, these groups are excluded from job level aggregations, and also do not count as part of the paid population.*

The trend of **inverse correlation between salary and the percentage of women** results in a lower average salary for women overall. This makes sense intuitively, and can also be shown mathematically (proof available upon request). 

The above two attributes, programming experience and job level, are just two examples at the tip of the iceberg. There may be other attributes for which there is such an inverse correlation between salary and percentage of women. Our purpose here is not to exhaust all attributes, but to demonstrate that this effect indeed contributes significantly to the gender pay gap.

But this effect is not the only factor. It is further exacerbated by plain old gender disparity in pay, as we shall see next. 

<section id="factor2">
<h1>Factor two: Women paid less than peers</h1>
</section>

Even when controlled for background and experience, women are paid less than their peers. 

In [ ]:
### Functions for plotting women vs non-women salary comparisons by background/experience

def salary_controlled_by(ax, col_name, col_title, row_order=None): 
    '''
    Plot salary controlled by a given column. 
    '''
    
    # obtain salary by group and gender
    df = us.groupby([col_name, 'is_woman']).salary.mean().to_frame().reset_index()\
        .pivot(index=col_name, columns='is_woman', values='salary')
    if row_order==None:
        df = df.sort_index()
    elif row_order=='salary':
        df = df.sort_values(False)
    else: 
        df = df.reindex(row_order)
    df.dropna(how='all').plot.barh(ax=ax, cmap='Paired')
    
    # configure plot
    ax.set_title('Salary by ' + col_title, fontsize=15)
    ax.set_xlabel('USD')
    ax.set_xticklabels([str(int(x/1000))+'k' for x in ax.get_xticks()])
    ax.set_ylabel('')
    ax.set_yticklabels([multi_line(x.get_text()) for x in ax.get_yticklabels()])
    ax.legend(['Non-women', 'Women'])
    
def multi_line(s, n=30): 
    '''
    Helper function for splitting axis labels into multiple lines. 
    '''
    words = s.split(' ')
    line = ''
    t = ''
    for w in words[:-1]: 
        line += w + ' '
        if len(line) >= n: 
            t += line[:-1] + '\n'
            line = ''
    t += line + words[-1]
    return t

This is true for the aforementioned attributes of programming experience and job level: 

In [ ]:
### Plot salary for woman/non-woman by programming experience (Q6) and job level (Q5)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

salary_controlled_by(ax1, 'Q6', 'Programming Experience (Q6)', experience_ordered)
salary_controlled_by(ax2, 'job_level', 'Job Level (Q5)', level)

plt.subplots_adjust(wspace=0.5)    # adjust spacing between subplots

plt.show()

The key takeaway from these bar plots is that, among every pair of bars that corresponds to the same attribute, the salary of women (brown bar) is lower than that of non-women (blue bar). 

The same can be observed for several other single-answer multiple choice questions, such as age (survey Q1), education (Q4), company size (Q20), data science team size (Q21), personal machine learning experience (Q15), and company machine learning in corporation (Q22). Again, the goal of this study is not to exhaustively check for gender pay disparity in every possible aspect, but to establish by example that it is indeed a prominant factor. 

In [ ]:
### Plot salary by single-answer multiple choice question attributes

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2, figsize=(15,18))

salary_controlled_by(ax1, 'Q1', 'Age (Q1)', sorted(us.Q1.unique()))
salary_controlled_by(ax2, 'Q4', 'Education (Q4)', 'salary')
salary_controlled_by(ax3, 'Q20', 'Company Size (Q20)', sorted(us.Q20.unique(), key=lambda x: len(str(x))))
salary_controlled_by(ax4, 'Q21', 'Data Science Team Size (Q21)', sorted(us.Q21.dropna().unique(), key=lambda x: int(x.replace('+','').split('-')[0])))
salary_controlled_by(ax5, 'Q15', 'Machine Learning Experience (Q15)', ['I do not use machine learning methods', 'Under 1 year', '1-2 years', '2-3 years', '3-4 years', '4-5 years', '5-10 years', '10-20 years', '20 or more years'])
salary_controlled_by(ax6, 'Q22', 'Company Machine Learning Incorporation (Q22)', 'salary')

plt.subplots_adjust(wspace=1, hspace=0.5)    # adjust spacing between subplots

plt.show()

The only two exceptions are the age group of 18-21, and those with 10-20 years of machine learning experience, which only account for a small fraction of the population (5.36% and 2.81%, respectively). 

In [ ]:
### Calculate percent of population for exceptions
print( 'Percentage of population with age 18-21:', round(us[ us.Q1=='18-21' ].shape[0] / us.shape[0] * 100, 2) )
print( 'Percentage of population with 10-20 years of machine learning experience:', round(us[ us.Q15=='10-20 years'].shape[0] / us.shape[0] * 100, 2) )

As a side note, a couple of additional observations can be made from the bar plots above: 

- Looking at the salary values (i.e., the absolute length of bars), we see that for all of the above attributes besides age, the salary increases along the y-axis for non-women (blue bars). Specifically, the salary for non-women generally increases with greater programming experience, higher job levels, higher education levels, bigger companies, bigger data science teams, more years of machine learning experience, and greater incorporation of machine learning by the company. However, the same trend does not always hold for the salary of woman (brown bars). This indicates that **considerations affecting the salary of women may be different, and perhaps more complicated, than that of non-women**. 

- Looking at the the women-to-non-women salary ratios (i.e., the relative lengths of brown and blue bars), we see from the second row of figures that **gender disparity in salary is lowest in mid-sized companies** (those with 250-999 or 50-249 employees, and those with 5-9 or 3-4 data scientists). 

- The first figure, salary by age group, may be supporting evidence for what is known as the "**motherhood penalty**". Women in their thirties and forties are paid significantly less than their peers, compared with women aged 25-29. Perhaps it is of no coincidence that the average age of first-time mothers with a college degree or higher is 30.3 years old, according to a [New York Times study published in 2018](https://www.nytimes.com/interactive/2018/08/04/upshot/up-birth-age-gap.html). 

Could gender pay gap be related to the difference in actual skill level between women and non-women? As a proxy for skill level, we count the number of technical tools each respondent uses, as surveyed in Q7, Q12-13, Q16-19, and Q31. 

Each bar plot below corresponds to one technical tool. There are many plots, but they all reveal the same message: Women are generally paid less than peers who use the same number of technical tools. 

In [ ]:
### Count the number of tools selected in the following list of questions

# Note: "None" (which is always the second-to-last option) corresponding to a count of 0; "Other" (always the last option) counts as 1 if the value is not NA. 

for q in ['Q7', 'Q12', 'Q14', 'Q16', 'Q17', 'Q18', 'Q19', 'Q31']: 
    us[ q + '_count' ] = us[ [x for x in questions.index if x.split('_')[0]==q] ].apply(lambda x:  x[:-2].count() + (not pd.isna(x[-1])) if pd.isna(x[-2]) else 0, axis=1)

In [ ]:
### Plot salary by number of tools used

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(nrows=3, ncols=3, figsize=(25,20))

salary_controlled_by(ax1, 'Q7_count', '# Programming Languages Used (Q7)')
salary_controlled_by(ax2, 'Q12_count', '# Specialized Hardware Used (Q12)')
salary_controlled_by(ax3, 'Q14_count', '# Visualization Libraries Used (Q14)')
salary_controlled_by(ax4, 'Q13', 'TPU Usage (Q13)', ['Never', 'Once', '2-5 times', '6-25 times', 'More than 25 times'])
salary_controlled_by(ax5, 'Q16_count', '# Machine Learning Frameworks Used (Q16)')
salary_controlled_by(ax6, 'Q17_count', '# Machine Learning Algorithms Used (Q17)')
salary_controlled_by(ax7, 'Q18_count', '# Computer Vision Methods Used (Q18)')
salary_controlled_by(ax8, 'Q19_count', '# Natural Language Processing Methods Used (Q19)')
salary_controlled_by(ax9, 'Q31_count', '# Business Intelligence Tools Used (Q31)')

plt.subplots_adjust(wspace=0.25, hspace=0.25)    # adjust spacing between subplots

plt.show()

The four exceptions, listed as follows, account for only a small number of the 2236 respondents: 
- Number of machine learning frameworks used (Q16) = 9 (3 respondents)
- Number of business intelligence tools used (Q31) = 14 (1 respondent), 13 (6 respondents), 6 (14 respondents)

In [ ]:
# Calculate number of respondents for exceptions
us.shape[0], (us.Q16_count == 9).sum(), (us.Q31_count == 14).sum(), (us.Q31_count == 13).sum(), (us.Q31_count == 6).sum()

<section id="mitigate">
<h1>Mitigating the gender pay gap</h1>
</section>

The actual cause and remedy for the gender pay gap is beyond the scope of this study. However, the observations above offer some possible directions. We have shown that the gender pay gap for 2020 US Kagglers can be decomposed into two factors. Accordingly, there are two corresponding approaches for mitigating the gender pay gap:

1. **Increase the percentage of women with higher-paying background and experience.** This could be achieved, for example, by offering more support to retain these women in the workforce, or by giving equal opportunities for their promotion to senior-level positions. 

2. **Decrease salary disparity between women and non-women with comparable background and experience.** For example, companies could establish a more objective metric for determining salary that is based on merit, so as to minimize gender bias. 



<section id="future">
<h1>Recommendations for future Kaggle surveys</h1>
</section>

Note that we have purposedly steered away from using terms such as "inequity" when describing the factors contributing to the gender pay gap. Inequity or unfair treatment of women may be playing a role here, but such a claim cannot be supported or refuted by data available from the survey. 

That said, it would be helpful if addition information could be collected in future Kaggle surveys. Below are some suggested survey questions that seem relatively easy to implement, and would provide further insights on factors contributing to the gender pay gap (as well as support other types of interesting analysis on the state of machine learning and data science): 

- **Greater accuracy in annual salary:** This would greatly improve any analysis related to pay. Some options include: offering more survey choices with greater granularity; separating $0 as its own option; allowing respondents to use their local currency (rather than converting to USD); asking respondents to round down their salary to the nearest thousand (say) and enter the actual number, rather than selecting from a list of given ranges. 

- **Hours of work per week:** This would help identify whether the respondent is in a full-time or part-time position. According to the [US Department of Labor](https://www.dol.gov/agencies/wb/data/latest-annual-data/full-and-part-time-employment), the majority of part-time workers are women. Thus, a detailed study of the gender pay gap requires controlling for the number of work hours per week, rather than just looking at the annual salary. 

- **Hourly wage:** This, along with the hours of work per week, provide an alternative measure of pay that may be more informative than the annual salary. It can account for part-time positions, internships, or freelance consulting projects. 

- **Number of years employed in ML/DS positions:** This is a more concrete measure of actual work experience than the current Q6 (years in programming) or Q15 (years using ML methods). For example, a respondent might wonder: If I've been programming since college but took a couple years off to raise children or complete an MBA, do those years count towards my programming years? This ambiguity can be avoided by counting the actual number of work years.

- **Academic discipline of, and years since, most recent degree earned:** Machine learning and data science are relatively new fields, and many people transition into such positions by either pursuing informal training on their own, or formally completing a degree or specialized program. Understanding the type of degree and the years since its completion would offer insights on whether women and non-women take different approaches in entering the field, and how that affects their pay. 